In [1]:
import os

os.chdir('../..')
print(os.getcwd())

/workspace/LLaSA


In [2]:
llava_processor = LlavaNextVideoProcessor.from_pretrained("/workspace/LLaSA/checkpoints/LLaVA-NeXT-Video-7B-hf", cache_dir='/workspace/LLaSA/checkpoints')


NameError: name 'LlavaNextVideoProcessor' is not defined

In [83]:
type(llava_processor)

transformers.models.llava_next_video.processing_llava_next_video.LlavaNextVideoProcessor

In [62]:
processor

LlavaNextVideoProcessor:
- video_processor: LlavaNextVideoImageProcessor {
  "crop_size": {
    "height": 336,
    "width": 336
  },
  "do_center_crop": true,
  "do_convert_rgb": true,
  "do_normalize": true,
  "do_pad": true,
  "do_rescale": true,
  "do_resize": true,
  "image_grid_pinpoints": [
    [
      336,
      672
    ],
    [
      672,
      336
    ],
    [
      672,
      672
    ],
    [
      1008,
      336
    ],
    [
      336,
      1008
    ]
  ],
  "image_mean": [
    0.48145466,
    0.4578275,
    0.40821073
  ],
  "image_processor_type": "LlavaNextVideoImageProcessor",
  "image_std": [
    0.26862954,
    0.26130258,
    0.27577711
  ],
  "processor_class": "LlavaNextVideoProcessor",
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 336
  }
}

- image_processor: LlavaNextImageProcessor {
  "crop_size": {
    "height": 336,
    "width": 336
  },
  "do_center_crop": true,
  "do_convert_rgb": true,
  "do_normalize": true,
 

In [5]:
# reload module 
import importlib
import torch
from transformers import BitsAndBytesConfig, LlavaNextVideoForConditionalGeneration
from model.llasa_arch import LLaSA
from model.llasa_processor import LlasaProcessor, LlavaNextVideoProcessor


importlib.reload(importlib.import_module("model.llasa_arch"))
importlib.reload(importlib.import_module("model.llasa_processor"))

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

processor = LlavaNextVideoProcessor.from_pretrained("/workspace/LLaSA/checkpoints/LLaVA-NeXT-Video-7B-hf", cache_dir='/workspace/LLaSA/checkpoints')
model = LLaSA.from_pretrained(
    "/workspace/LLaSA/checkpoints/LLaVA-NeXT-Video-7B-hf",
    quantization_config=quantization_config,
    device_map='auto',
    cache_dir = '/workspace/LLaSA/checkpoints'
)


None


TypeError: Received a NoneType for argument tokenizer, but a ('LlamaTokenizer', 'LlamaTokenizerFast') was expected.

In [ ]:
!pip install --upgrade -q accelerate bitsandbytes
! pip install protobuf

In [37]:
from transformers import BitsAndBytesConfig, LlavaNextVideoForConditionalGeneration, LlavaNextVideoProcessor
import torch
from model.llasa_arch import LLaSA
from model.llasa_processor import LlasaProcessor

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

processor = LlasaProcessor.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf", cache_dir='/workspace/LLaSA/checkpoints')
model = LLaSA.from_pretrained(
    "llava-hf/LLaVA-NeXT-Video-7B-hf",
    quantization_config=quantization_config,
    device_map='auto',
    cache_dir = '/workspace/LLaSA/checkpoints'
)

Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}
Loading checkpoint shards:   0%|          | 0/3 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
model

In [3]:
import av
import numpy as np

def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.

    Args:
        container (av.container.input.InputContainer): PyAV container.
        indices (List[int]): List of frame indices to decode.

    Returns:
        np.ndarray: np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

In [4]:
from huggingface_hub import hf_hub_download

# Download video from the hub
#video_path_1 = hf_hub_download(repo_id="raushan-testing-hf/videos-test", filename="sample_demo_1.mp4", repo_type="dataset")
video_path_2 = hf_hub_download(repo_id="raushan-testing-hf/videos-test", filename="karate.mp4", repo_type="dataset")

video_path_1 = '/workspace/LLaSA/dataset/A2D/clips320H/__KkKB4wzrY.mp4'

container = av.open(video_path_1)

# sample uniformly 8 frames from the video (we can sample more for longer videos)
total_frames = container.streams.video[0].frames
indices = np.arange(0, total_frames, total_frames / 8).astype(int)
clip_baby = read_video_pyav(container, indices)


container = av.open(video_path_2)

# sample uniformly 8 frames from the video (we can sample more for longer videos)
total_frames = container.streams.video[0].frames
indices = np.arange(0, total_frames, total_frames / 8).astype(int)
clip_karate = read_video_pyav(container, indices)

In [5]:
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# np array with shape (frames, height, width, channels)
video = clip_baby

fig = plt.figure()
im = plt.imshow(video[0,:,:,:])

plt.close() # this is required to not display the generated image

def init():
    im.set_data(video[0,:,:,:])

def animate(i):
    im.set_data(video[i,:,:,:])
    return im

#anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video.shape[0],
#                               interval=100)
#HTML(anim.to_html5_video())

In [6]:
# Lets also load 2 images for generation from image data

from PIL import Image
import requests

image_stop = Image.open(requests.get("https://www.ilankelman.org/stopsigns/australia.jpg", stream=True).raw)
image_snowman = Image.open(requests.get("https://huggingface.co/microsoft/kosmos-2-patch14-224/resolve/main/snowman.jpg", stream=True).raw)

In [7]:
# Each "content" is a list of dicts and you can add image/video/text modalities
conversation = [
      {
          "role": "user",
          "content": [
              {"type": "text", "text": "Why is this video funny?"},
              {"type": "video"},
              ],
      },
]

conversation_2 = [
      {
          "role": "user",
          "content": [
              {"type": "text", "text": "What do you see in this video?"},
              {"type": "video"},
              ],
      },
]

prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
prompt_2 = processor.apply_chat_template(conversation_2, add_generation_prompt=True)


In [8]:
inputs = processor([prompt, prompt_2], videos=[clip_baby, clip_karate], padding=True, return_tensors="pt").to(model.device)

Expanding inputs for image/video tokens in LLaVa-NeXT-Video should be done in processing. Please add `patch_size` and `vision_feature_select_strategy` to the model's processing config or set directly with `processor.patch_size = {{patch_size}}` and processor.vision_feature_select_strategy = {{vision_feature_select_strategy}}`. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.


None None {} dict_keys(['pixel_values_videos'])
dict_keys(['input_ids', 'attention_mask'])


In [9]:
inputs = processor([prompt], videos=[clip_baby], padding=True, return_tensors="pt").to(model.device)

None None {} dict_keys(['pixel_values_videos'])
dict_keys(['input_ids', 'attention_mask'])


In [51]:
inputs

{'input_ids': tensor([[    1,  3148,  1001, 29901, 29871, 32000,    13, 11008,   338,   445,
          4863,  2090,  1460, 29973,   319,  1799,  9047, 13566, 29901]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0'), 'pixel_values_videos': tensor([[[[[ 9.9604e-01,  9.9604e-01,  9.9604e-01,  ...,  3.8290e-01,
             3.8290e-01,  3.8290e-01],
           [ 1.0106e+00,  1.0106e+00,  1.0106e+00,  ...,  3.8290e-01,
             3.8290e-01,  3.8290e-01],
           [ 1.0106e+00,  1.0106e+00,  1.0106e+00,  ...,  3.8290e-01,
             3.8290e-01,  3.8290e-01],
           ...,
           [-1.6901e+00, -1.6901e+00, -1.6901e+00,  ..., -1.7923e+00,
            -1.7923e+00, -1.7923e+00],
           [-1.6901e+00, -1.6901e+00, -1.6901e+00,  ..., -1.7923e+00,
            -1.7923e+00, -1.7923e+00],
           [-1.6901e+00, -1.6901e+00, -1.6901e+00,  ..., -1.7923e+00,
            -1.7923e+00, -1.7923e+00]],

  

In [49]:
inputs['pixel_values_videos'].shape

torch.Size([1, 8, 3, 336, 336])

In [54]:
generate_kwargs = {"max_new_tokens": 100, "do_sample": True, "top_p": 0.9}



output = model.generate(**inputs, **generate_kwargs)
generated_text = processor.batch_decode(output, skip_special_tokens=False)

legacy
tensor([[    1,  3148,  1001, 29901, 29871, 32000,    13, 11008,   338,   445,
          4863,  2090,  1460, 29973,   319,  1799,  9047, 13566, 29901]],
       device='cuda:0')
tensor([[    1,  3148,  1001, 29901, 29871, 32000,    13, 11008,   338,   445,
          4863,  2090,  1460, 29973,   319,  1799,  9047, 13566, 29901]],
       device='cuda:0')
feature dim and lens: torch.Size([1152, 4096]) tensor([1152], device='cuda:0')
torch.Size([1, 1170, 4096]) torch.Size([1, 1170]) torch.Size([1, 1170]) None torch.Size([1, 1170])
input embed: torch.Size([1, 1170, 4096])


AttributeError: 'CausalLMOutputWithPast' object has no attribute 'shape'

In [45]:
print(generated_text)

["<s> USER: <video>\nWhy is this video funny? ASSISTANT: The humor in this video comes from the unexpected and somewhat anthropomorphic interaction between a bird (seagull) and a person. In the video, a seagull takes something from a person's hand while that person is standing, which can be a surprising and amusing sight as it challenges the viewer's expectations of how humans and wildlife might interact. The seagull's boldness in snatching something from a human, possibly food, creates a"]


In [46]:
print(processor)

LlavaNextVideoProcessor:
- video_processor: LlavaNextVideoImageProcessor {
  "crop_size": {
    "height": 336,
    "width": 336
  },
  "do_center_crop": true,
  "do_convert_rgb": true,
  "do_normalize": true,
  "do_pad": true,
  "do_rescale": true,
  "do_resize": true,
  "image_grid_pinpoints": [
    [
      336,
      672
    ],
    [
      672,
      336
    ],
    [
      672,
      672
    ],
    [
      1008,
      336
    ],
    [
      336,
      1008
    ]
  ],
  "image_mean": [
    0.48145466,
    0.4578275,
    0.40821073
  ],
  "image_processor_type": "LlavaNextVideoImageProcessor",
  "image_std": [
    0.26862954,
    0.26130258,
    0.27577711
  ],
  "processor_class": "LlavaNextVideoProcessor",
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 336
  }
}

- image_processor: LlavaNextImageProcessor {
  "crop_size": {
    "height": 336,
    "width": 336
  },
  "do_center_crop": true,
  "do_convert_rgb": true,
  "do_normalize": true,
 

In [43]:
print(model.config)

LlavaNextVideoConfig {
  "_name_or_path": "llava-hf/LLaVA-NeXT-Video-7B-hf",
  "architectures": [
    "LlavaNextVideoForConditionalGeneration"
  ],
  "ignore_index": -100,
  "image_grid_pinpoints": [
    [
      336,
      672
    ],
    [
      672,
      336
    ],
    [
      672,
      672
    ],
    [
      1008,
      336
    ],
    [
      336,
      1008
    ]
  ],
  "image_seq_length": 576,
  "image_token_index": 32001,
  "model_type": "llava_next_video",
  "projector_hidden_act": "gelu",
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "float16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "spatial_pool_

In [ ]:
!pip install h5py

In [ ]:

from scipy.io import loadmat

# Load the .mat file
data = loadmat()

In [ ]:
import h5py
import torch
import numpy as np

# Open the .mat file
with h5py.File('/workspace/LLaSA/dataset/A2D/a2d_annotation_with_instances/ouFmCnCh1wk/00030.h5', 'r') as f:
    # List all variables stored in the .mat file
    print("Keys in the file:", list(f.keys()))


    def print_structure(name, obj):
        """Print the structure of the HDF5 file"""
        if isinstance(obj, h5py.Dataset):
            print(f"Dataset: {name}, Shape: {obj.shape}, Data type: {obj.dtype}")
        elif isinstance(obj, h5py.Group):
            print(f"Group: {name}")

    # Visit each object in the file
    f.visititems(print_structure)

    coord = f['reBBox'][:]
    print(f['instance'][:])



In [ ]:
!pip uninstall opencv-contrib-python -y
!pip uninstall opencv-python-headless -y 
!pip uninstall opencv-python -y


In [ ]:
!pip install --no-cache-dir opencv-contrib-python 
!pip install  --no-cache-dir opencv-python
!pip install --no-cache-dir opencv-python-headless


In [ ]:
!pip install --upgrade opencv-python

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def draw_bounding_box(image_path, bbox):
    """
    Retrieve an image from a local path and draw a bounding box on it using matplotlib.

    :param image_path: Path to the local image file
    :param bbox: Bounding box coordinates in the format (x_min, y_min, x_max, y_max)
    """
    # Load the image using PIL
    image = Image.open(image_path)

    # Create a figure and axis
    fig, ax = plt.subplots(1)

    # Display the image
    ax.imshow(image)

    # Extract bounding box coordinates
    x_min, y_min, x_max, y_max = bbox

    # Create a Rectangle patch
    rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                             linewidth=2, edgecolor='g', facecolor='none')

    # Add the rectangle to the plot
    ax.add_patch(rect)

    # Show the plot
    plt.show()

# Example usage
image_path = '/workspace/LLaSA/dataset/A2D/clips320jpeg/ouFmCnCh1wk/00030.jpeg'  # Replace with the path to your image file
num = 2
bbox = tuple(sub_coord[num] for sub_coord in coord)  # Example bounding box coordinates (x_min, y_min, x_max, y_max)

draw_bounding_box(image_path, bbox)


In [ ]:
[[112.7125      78.1         51.475      177.5        243.175
  313.2875     135.7875    ]
 [136.88888889 100.44444444 158.22222222 159.11111111 110.22222222
   88.         156.44444444]
 [165.9625      98.5125      79.875      234.3        265.3625
  326.6        146.4375    ]
 [205.33333333 164.44444444 251.55555556 240.88888889 168.
  104.         164.44444444]]